In [1]:
import torch

def transform_tensor(X, weights):
    # X is a 4D tensor of shape (a, b, c, d)
    # weights is a 1D tensor of shape (2,)
    
    a, b, c, d = X.shape
    # 스택하고 리쉐이프하는 과정
    X_stacked = X.permute(1, 2, 0, 3).reshape(b, c, a * d)
    
    # 새로운 텐서를 생성하는데, 마지막 차원을 계산에 이용
    new_last_dim = []
    for i in range(0, a * d, 2):  # 2개씩 짝을 지어 처리
        # 인접한 값들을 가중 평균
        weighted_avg = (weights[0] * X_stacked[:, :, i] + weights[1] * X_stacked[:, :, i+1]) / weights.sum()
        new_last_dim.append(weighted_avg)
    
    # 새로운 마지막 차원을 스택
    new_tensor = torch.stack(new_last_dim, dim=-1)
    return new_tensor

# 예시 텐서
X = torch.tensor([[[[1, 2], [3, 4]], [[1, 2], [3, 4]]], [[[5, 6], [7, 8]], [[5, 6], [7, 8]]]])
weights = torch.tensor([1, 2])

In [12]:
X

tensor([[[[1, 2],
          [3, 4]],

         [[1, 2],
          [3, 4]]],


        [[[5, 6],
          [7, 8]],

         [[5, 6],
          [7, 8]]]])

In [18]:
X_stacked = X.permute(1, 2, 0, 3).reshape(2, 2, 4)
X_stacked

tensor([[[1, 2, 5, 6],
         [3, 4, 7, 8]],

        [[1, 2, 5, 6],
         [3, 4, 7, 8]]])

In [25]:
# 부드럽게 Tensor 연결하는 함수 정의
def smoothing(X):
    # X is a 4D tensor of shape (a, b, c, d)
    # weights is a 1D tensor of shape (2,)
    
    a, b, c, d = X.shape
    # 스택하고 리쉐이프하는 과정
    X_stacked = X.permute(1, 2, 0, 3).reshape(b, c, a * d)
    
    # 2개씩 짝을 지어 처리
    for j in range(a-1):  
        ct_point = d*(j+1)

        ct_before = X_stacked[:, :, ct_point-1]
        ct_after = X_stacked[:, :, ct_point]
        for k in range(d//2):
            X_stacked[:, :, k + ct_point - d//2] = (d-k)/(d+1) * X_stacked[:, :, k + ct_point - d//2] + (k+1)/(d+1) * ct_after
        for k in range(d//2):
            X_stacked[:, :, k + ct_point ] = (d//2 + k +1)/(d+1) * X_stacked[:, :, k + ct_point ] + (d//2 - k)/(d+1) * ct_before
    
    return X_stacked

In [21]:
ct_point = 2
ct_before = X_stacked[:, :, ct_point-1]
ct_after = X_stacked[:, :, ct_point]
ct_before, ct_after

(tensor([[2, 4],
         [2, 4]]),
 tensor([[5, 7],
         [5, 7]]))

In [22]:
X_stacked[:, :,1] = 2/3 * X_stacked[:, :, 1] + 1/3 * ct_after

In [23]:
X_stacked[:, :,2] = 2/3 * X_stacked[:, :, 2] + 1/3 * ct_before

In [24]:
X_stacked

tensor([[[1, 3, 4, 6],
         [3, 5, 6, 8]],

        [[1, 3, 4, 6],
         [3, 5, 6, 8]]])